## Sandbox
A space for free experiments and building improvements for Zoho Api handling

In [ ]:
# setting notebook source
with open('./config_path.py') as f:
    exec(f.read())

In [ ]:
from config import secrets, settings
from zoho_api import *
import pandas as pd
import copy
from dateutil import parser
from datetime import date, time
import requests

In [ ]:
backup_data = {}
response_data = {}

In [ ]:
obtain_token()

In [ ]:
refresh_token()

In [ ]:
SLAs = {}
SLAs['SupportME Premium'] = '10977000006774075'
SLAs['SupportME Comfort'] = '10977000006804083'
SLAs['SupportME ComfortPlus'] = '10977000006804083'
SLAs['SupportME Standard'] = '10977000006804131'
SLAs[None] = ''
SLAs

In [ ]:
SupportHours = {}
SupportHours['SupportME Premium'] = 204
SupportHours['SupportME Comfort'] = 96
SupportHours['SupportME ComfortPlus'] = 8
SupportHours['SupportME Standard'] = 36
SupportHours[None] = None
SupportHours

## Tests

In [ ]:
def list_tags(limit=None, departmentName="Technical support"):
    # If limit is None, fetch all tags
    depts, _ = list_departments()
    deptId = depts[departmentName]

    headers = {
        'Authorization': f'Zoho-oauthtoken {secrets.access_token}',
        'orgId': secrets.org_id
    }

    tag_data = {}
    index = 0
    page_limit = 100 if limit is None else min(limit, 100)  # use 100 max per page

    while True:
        response = requests.get(
            f'{settings.api_url}/ticketTags?limit={page_limit}&departmentId={deptId}&from={index}',
            headers=headers
        )

        if response.status_code != 200:
            print(f'Błąd: {response.status_code}')
            print(response.text)
            return None

        data = response.json()
        tags_cleaned = {record['id']: record['name'] for record in data['data']}
        tag_data.update(tags_cleaned)

        # Stop if fewer results were returned than requested (end of data)
        if len(data['data']) < page_limit:
            break

        index += page_limit

        # Stop if total collected tags reach the specified limit
        if limit is not None and len(tag_data) >= limit:
            tag_data = dict(list(tag_data.items())[:limit])
            break

    return tag_data

In [ ]:
tags = list_tags(limit=250)
len(tags)

In [ ]:
sorted_tags = dict(sorted(tags.items(), key=lambda item: item[1]))
sorted_tags

In [ ]:
def list_threads(ticket_id: str, index: int, limit = 100):

    # Nagłówki żądania
    thread_ids = []
    thread_data = []
    headers = {
        'Authorization': f'Zoho-oauthtoken {secrets.access_token}',
        'orgId': secrets.org_id
    }
    # Wykonanie żądania GET do API
    response = requests.get(
        f'{settings.api_url}/tickets/{ticket_id}/conversations',
        headers=headers
    )
    # Sprawdzenie odpowiedzi
    if response.status_code == 200:
        data = response.json()
        thread_data.extend(data['data'])
        for thread in thread_data:
            thread_ids.append(thread['id'])
    else:
        print(f'Błąd: {response.status_code}')
        print(response.text)
    return thread_data, thread_ids

In [ ]:
def read_thread(ticket_id: str, thread_id: str):
    # Nagłówki żądania
    headers = {
        'Authorization': f'Zoho-oauthtoken {secrets.access_token}',
        'orgId': secrets.org_id
    }
    
    # Wykonanie żądania GET do API
    response = requests.get(
        f'{settings.api_url}/tickets/{ticket_id}/threads/{thread_id}',
        headers=headers
    )
    
    # Sprawdzenie odpowiedzi
    if response.status_code == 200:
        thread_data = response.json()
        return thread_data, response
    else:
        print(f'Błąd: {response.status_code}')
        print(response.text)
        return None, response
    

In [ ]:
response, ids = list_threads(ticket_id="10977000061152288",index=0)

In [ ]:
ids

In [ ]:
thread, response = read_thread(ticket_id="10977000061152288",thread_id='10977000061132463')

In [ ]:
thread['content']

In [ ]:
ids

In [ ]:
from bs4 import BeautifulSoup
import re


# Parsowanie HTML za pomocą BeautifulSoup
soup = BeautifulSoup(thread['content'], 'html.parser')

# Wyciągnięcie czystego tekstu
text_content = soup.get_text()
print (text_content)
# Wzorce do usunięcia podpisów, kontaktów i historii konwersacji
signature_patterns = [
    r'Pozdrawiamy,',
    r'Dział Wsparcia Sprzedaży',
    r'tel\\.\\s*\\+?\\d+',
    r'From:.*',
    r'Sent:.*',
    r'--+'
]

# Usunięcie wszystkiego po pierwszym wystąpieniu któregokolwiek z wzorców
for pattern in signature_patterns:
    text_content = re.split(pattern, text_content)[0]

# Zapis oczyszczonego tekstu do pliku
with open('cleaned_email.txt', 'w', encoding='utf-8') as file:
    file.write(text_content.strip())

print("Oczyszczona treść wiadomości została zapisana do 'cleaned_email.txt'.")
